### **How trustworthy is your local representative?**

#### Data Sources:
politifact api documentation: http://static.politifact.com/api/v2apidoc.html <br>
govtrack api documentation: https://www.govtrack.us/developers/api <br>
propublica: https://projects.propublica.org/api-docs/congress-api/ <br>
congressional misconduct database: https://www.govtrack.us/misconduct <br>
news api: https://newsapi.org/docs <br>
votesmart: https://github.com/votesmart/python-votesmart/tree/master

### To do: April 15

1. Crawl truth-o-meter data
2. Join truth-o-meter, misconduct db, and propublica
3. Structure bill vote
4. Incorporate news headlines
5. incorprate tweets (?)


In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import clear_output

In [2]:
data_path = '/Users/Xie/Documents/School/Senior Spring/PPDS/Data/'
pd.read_csv(data_path+'candidates_2016_1103.csv').head()

,fec_candidate_id,name,party,seat,status,address_one,address_two,city,state,zip,fec_committee_id,cycle,district,office_state,cand_status,branch,crp_id,clean_name,google_entity_id
0,H0PA07082,"MEEHAN, PATRICK L. MR.",REP,NaN,I,1029 EDMONDS AVENUE,NaN,DREXEL HILL,PA,19026.0,C00466870,2016,7,PA,C,H,N00031134,Patrick Meehan,/m/03m4jty
1,H4LA01147,"DUGAS, LEE A",DEM,NaN,C,908 MINNESOTA AVE,NaN,KENNER,LA,70062.0,NaN,2016,1,LA,N,H,NaN,Lee A. Dugas,NaN
2,H4LA02087,"DAVENPORT, SAMUEL",LIB,NaN,C,3127 COLLEGE CT,APT B,NEW ORLEANS,LA,70125.0,NaN,2016,2,LA,N,H,NaN,Samuel Davenport,NaN
3,H2GU00033,"BORDALLO, MADELEINE Z. MS.",DEM,NaN,I,P.O. BOX 2448,NaN,HAGATNA,GU,96932.0,C00369686,2016,0,GU,C,H,N00024866,Madeleine Z. Bordallo,/m/01s_xp
4,H6AL04098,"ADERHOLT, ROBERT BROWN",REP,NaN,I,PO BOX 1158,NaN,HALEYVILLE,AL,35565.0,C00313247,2016,4,AL,C,H,N00003028,Robert Aderholt,/m/024p03


### Politifact
politifact api documentation: http://static.politifact.com/api/v2apidoc.html 

In [3]:
# Politifact truth-o-meter data

pf_url = 'http://www.politifact.com/api/v/2/statement/'

limit = 50
offset = 0

params = {
    'format' : 'json',
    'edition__edition_slug':'truth-o-meter',
    'limit' : limit,
    'offset' : offset,
    'order_by' : 'ruling_date',
}

resp = requests.get(pf_url,params=params).json()
pf = pd.DataFrame(resp['objects'])

speaker = pf['speaker']

In [4]:
politifact = pf[['ruling','speaker','statement',
         'statement_context','statement_date',
         'statement_type','subject','sources']]

for i in politifact.index:
  
    politifact.loc[i,'ruling'] = politifact.loc[
        i,'ruling']['ruling_slug']
    
    politifact.loc[i,'speaker'] = politifact.loc[
        i,'speaker']['name_slug']
    
    politifact.loc[i,'statement_type'] = politifact.loc[
        i,'statement_type']['statement_type']
    
    politifact.loc[i,'statement'] = (
        politifact.loc[i,'statement']
        .replace('<p>"','').replace('"</p>','')
        .replace('<p>&quot;','').replace('&quot;</p>\r\n',''))
    
    politifact.loc[i,'subject'] = [
        subject['subject_slug'] 
        for subject in politifact.loc[i,'subject']]
    
    politifact.loc[i,'statement_date'] = pd.to_datetime(
        politifact.loc[i,'statement_date']).date()

clear_output()

In [5]:
politifact.head(10)

,ruling,speaker,statement,statement_context,statement_date,statement_type,subject,sources
0,true,tommy-thompson,Tommy Thompson created the first school choice...,his campaign Web site.,2007-05-01,Claim,[education],"<p>Krista Kafer, Senior Policy Analyst for Edu..."
1,true,tommy-thompson,"""Tommy Thompson is the father of welfare refor...",the Web.,2007-05-01,Claim,[job-accomplishments],Interviews: Former Rep. Clay Shaw; Ron Haskins...
2,false,tom-tancredo,For every single scientist that tells you (glo...,"Columbia, S.C.",2007-05-15,Claim,"[climate-change, environment]","<p><a href=""http://www.ipcc.ch/"">Intergovernme..."
3,mostly-true,bill-richardson,"""New Mexico was 46th in teacher pay (when he w...",a TV ad.,2007-05-10,Claim,[education],"New Mexico Department of Labor,\n<a href=""http..."
4,true,john-edwards,"""We now have the greatest income inequality si...",a story by Reuters wire service.,2007-05-12,Claim,[economy],"<a href=""http://elsa.berkeley.edu/~saez/pikett..."
5,mostly-true,bill-richardson,"New Mexico moved ""up to"" sixth in the nation i...",a TV Ad.,2007-05-10,Claim,[job-accomplishments],"New Mexico Department of Labor,\n<a href=""http..."
6,half-true,barack-obama,"""Both China and Japan have surpassed us, with ...","Detroit, Mich.",2007-05-07,Claim,[energy],"<a href=""http://www.pewclimate.org/global-warm..."
7,half-true,barack-obama,"""If we increase fuel efficiency standards to 4...","Washington, DC.",2007-06-19,Claim,[energy],"<a href=""http://www.eia.doe.gov/pub/oil_gas/pe..."
8,true,sam-brownback,"""I'm for a three-state political solution in I...",,2007-06-08,Attack,[iraq],"<a href=""http://www.joebiden.com/issues/?id=00..."
9,mostly-true,bill-richardson,"""I used tax cuts to help create over 80,000 jo...",a TV Ad.,2007-05-10,Claim,[taxes],"New Mexico Department of Labor,\n<a href=""http..."


In [6]:
pd.DataFrame([entry for entry in speaker]).head()['party']

0    {'id': 3, 'party': 'Republican', 'party_slug':...
1    {'id': 3, 'party': 'Republican', 'party_slug':...
2    {'id': 3, 'party': 'Republican', 'party_slug':...
3    {'id': 1, 'party': 'Democrat', 'party_slug': '...
4    {'id': 1, 'party': 'Democrat', 'party_slug': '...
Name: party, dtype: object

### Govtrack (Deprecated)
govtrack api documentation: https://www.govtrack.us/developers/api 

In [7]:
# Data on legislators

person = 'https://www.govtrack.us/api/v2/person'
role = 'https://www.govtrack.us/api/v2/role'

params = {
    'current' : 'true'
}

resp = requests.get(person).json()

legis = pd.DataFrame(resp['objects'])
hillary = legis[legis['lastname']=='Clinton']['bioguideid'].iloc[0]

### Propublica
propublica: https://projects.propublica.org/api-docs/congress-api/ 

In [8]:
from congress import Congress
propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'
congress = Congress(propublica_key)

cong = 80 # Earliest
cong = 115 # Latest

senate = congress.members.filter('senate',congress = cong)
house = congress.members.filter('house', congress = cong)

senate_cols = ['id','lis_id','state','state_rank','party',
               'first_name','last_name','middle_name',
               'gender','date_of_birth','in_office','govtrack_id',
               'votes_with_party_pct','missed_votes_pct','missed_votes',
               'senate_class','next_election','title',
               'leadership_role']

house_cols = ['id','state','district','party',
              'first_name','last_name','middle_name',
              'gender','date_of_birth','in_office','govtrack_id',
              'votes_with_party_pct','missed_votes_pct','missed_votes',
              'next_election','title',
              'leadership_role','ideal_point']

senate = pd.DataFrame(senate[0]['members'])[senate_cols]
house = pd.DataFrame(house[0]['members'])[house_cols]

In [9]:
import requests
from lxml import html
import pandas as pd
import re

url = congress.votes.by_month('house')['votes'][0]['source']

response = requests.get(url)
doc = html.fromstring(response.content)

print(doc.xpath('//vote-desc')[0].text_content())
print(doc.xpath('//legis-num')[0].text_content())
print('\n'+'*'*20+'\n')

for record in doc.xpath('//vote-data/recorded-vote'):
    rep_id = record.xpath('./legislator')[0].get('name-id')
    party = record.xpath('./legislator')[0].get('party')
    state = record.xpath('./legislator')[0].get('state')
    name = record.xpath('./legislator')[0].text_content()
    vote = record.xpath('./vote')[0].text_content()
    
    print(name,party+'-'+state)
    print(rep_id)
    print(vote)
    print('-'*5)

Volcker Rule Regulatory Harmonization Act
H R 4790

********************

Abraham R-LA
A000374
Yea
-----
Adams D-NC
A000370
Yea
-----
Aderholt R-AL
A000055
Yea
-----
Aguilar D-CA
A000371
Yea
-----
Allen R-GA
A000372
Yea
-----
Amash R-MI
A000367
Yea
-----
Amodei R-NV
A000369
Yea
-----
Arrington R-TX
A000375
Yea
-----
Babin R-TX
B001291
Yea
-----
Bacon R-NE
B001298
Yea
-----
Banks (IN) R-IN
B001299
Yea
-----
Barletta R-PA
B001269
Yea
-----
Barr R-KY
B001282
Yea
-----
Barragán D-CA
B001300
Yea
-----
Barton R-TX
B000213
Yea
-----
Bass D-CA
B001270
Not Voting
-----
Beatty D-OH
B001281
Yea
-----
Bera D-CA
B001287
Yea
-----
Bergman R-MI
B001301
Yea
-----
Beyer D-VA
B001292
Yea
-----
Biggs R-AZ
B001302
Yea
-----
Bilirakis R-FL
B001257
Yea
-----
Bishop (GA) D-GA
B000490
Not Voting
-----
Bishop (MI) R-MI
B001293
Yea
-----
Bishop (UT) R-UT
B001250
Yea
-----
Black R-TN
B001273
Yea
-----
Blackburn R-TN
B001243
Yea
-----
Blum R-IA
B001294
Yea
-----
Blumenauer D-OR
B000574
Nay
-----
Blunt Rochester D

-----
Noem R-SD
N000184
Not Voting
-----
Nolan D-MN
N000127
Nay
-----
Norcross D-NJ
N000188
Nay
-----
Norman R-SC
N000190
Yea
-----
Nunes R-CA
N000181
Yea
-----
O'Halleran D-AZ
O000171
Yea
-----
O'Rourke D-TX
O000170
Yea
-----
Olson R-TX
O000168
Yea
-----
Palazzo R-MS
P000601
Yea
-----
Pallone D-NJ
P000034
Nay
-----
Palmer R-AL
P000609
Yea
-----
Panetta D-CA
P000613
Yea
-----
Pascrell D-NJ
P000096
Nay
-----
Paulsen R-MN
P000594
Yea
-----
Payne D-NJ
P000604
Nay
-----
Pearce R-NM
P000588
Yea
-----
Pelosi D-CA
P000197
Nay
-----
Perlmutter D-CO
P000593
Yea
-----
Perry R-PA
P000605
Yea
-----
Peters D-CA
P000608
Yea
-----
Peterson D-MN
P000258
Yea
-----
Pingree D-ME
P000597
Nay
-----
Pittenger R-NC
P000606
Yea
-----
Pocan D-WI
P000607
Nay
-----
Poe (TX) R-TX
P000592
Yea
-----
Poliquin R-ME
P000611
Yea
-----
Polis D-CO
P000598
Nay
-----
Posey R-FL
P000599
Yea
-----
Price (NC) D-NC
P000523
Nay
-----
Quigley D-IL
Q000023
Yea
-----
Raskin D-MD
R000606
Nay
-----
Ratcliffe R-TX
R000601
Yea
-----
R

In [10]:
import requests
from lxml import html
import pandas as pd
import re

url = congress.votes.by_month('senate')['votes'][0]['source']

response = requests.get(url)
doc = html.fromstring(response.content)

print(doc.xpath('//vote_title')[0].text_content())
print('\n'+'*'*20+'\n')

for member in doc.xpath('//member'):
    print(member.xpath('./member_full')[0].text_content())
    print(member.xpath('./first_name')[0].text_content())
    print(member.xpath('./vote_cast')[0].text_content())
    print(member.xpath('./lis_member_id')[0].text_content())
    print('-'*10)

Motion to Invoke Cloture Re: Rebecca Grady Jennings, of Kentucky, to be U.S. District Judge for the Western District of Kentucky

********************

Alexander (R-TN)
Lamar
Yea
S289
----------
Baldwin (D-WI)
Tammy
Yea
S354
----------
Barrasso (R-WY)
John
Yea
S317
----------
Bennet (D-CO)
Michael
Yea
S330
----------
Blumenthal (D-CT)
Richard
Yea
S341
----------
Blunt (R-MO)
Roy
Yea
S342
----------
Booker (D-NJ)
Cory
Yea
S370
----------
Boozman (R-AR)
John
Yea
S343
----------
Brown (D-OH)
Sherrod 
Yea
S307
----------
Burr (R-NC)
Richard
Yea
S300
----------
Cantwell (D-WA)
Maria
Yea
S275
----------
Capito (R-WV)
Shelley
Yea
S372
----------
Cardin (D-MD)
Ben
Yea
S308
----------
Carper (D-DE)
Thomas
Yea
S277
----------
Casey (D-PA)
Bob
Yea
S309
----------
Cassidy (R-LA)
Bill
Yea
S373
----------
Collins (R-ME)
Susan
Yea
S252
----------
Coons (D-DE)
Christopher
Not Voting
S337
----------
Corker (R-TN)
Bob
Yea
S310
----------
Cornyn (R-TX)
John
Yea
S287
----------
Cortez Masto (D-NV)
Catheri

In [11]:
# compare
congress.members.compare('S000033','C001098','senate',congress=cong)

{'agree_percent': 16.45,
 'chamber': 'Senate',
 'common_votes': 377,
 'congress': '115',
 'disagree_percent': 83.55,
 'disagree_votes': 315,
 'first_member_api_uri': 'https://api.propublica.org/congress/v1/members/S000033.json',
 'first_member_id': 'S000033',
 'second_member_api_uri': 'https://api.propublica.org/congress/v1/members/C001098.json',
 'second_member_id': 'C001098'}

In [12]:
#bills
pd.DataFrame(congress.bills.by_member(senate['id'][0])['bills']).head(3)

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,False,s2513-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,15,{'R': 15},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to improve school safety and mental hea...,None
1,False,s2509-115,s,https://api.propublica.org/congress/v1/115/bil...,Senate Energy and Natural Resources Committee,115,https://www.congress.gov/bill/115th-congress/s...,7,"{'R': 4, 'D': 2, 'I': 1}",None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to establish the National Park Restorat...,None
2,True,s2434-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,0,{},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,"A bill to amend the Federal Food, Drug, and Co...",None


In [13]:
# same thing, above is probably better
pd.DataFrame(congress.members.bills(senate['id'][0])['bills']).head(3)

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,False,s2513-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,15,{'R': 15},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to improve school safety and mental hea...,None
1,False,s2509-115,s,https://api.propublica.org/congress/v1/115/bil...,Senate Energy and Natural Resources Committee,115,https://www.congress.gov/bill/115th-congress/s...,7,"{'R': 4, 'D': 2, 'I': 1}",None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to establish the National Park Restorat...,None
2,True,s2434-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,0,{},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,"A bill to amend the Federal Food, Drug, and Co...",None


In [14]:
print(dir(congress.bills))

['BASE_URI', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'amendments', 'apikey', 'by_member', 'cosponsors', 'fetch', 'get', 'http', 'introduced', 'major', 'passed', 'recent', 'related', 'subjects', 'upcoming', 'updated']


In [15]:
pd.DataFrame(congress.bills.subjects('s2513',congress=cong))['subjects'].head()

0     {'name': 'Elementary and secondary education',...
1     {'name': 'Congressional oversight', 'url_name'...
2     {'name': 'Crime prevention', 'url_name': 'crim...
3     {'name': 'Child health', 'url_name': 'child-he...
4     {'name': 'Building construction', 'url_name': ...
5     {'name': 'Detention of persons', 'url_name': '...
6     {'name': 'School administration', 'url_name': ...
7     {'name': 'Mental health', 'url_name': 'mental-...
8     {'name': 'Educational facilities and instituti...
9     {'name': 'Violent crime', 'url_name': 'violent...
10    {'name': 'Education programs funding', 'url_na...
11    {'name': 'Teaching, teachers, curricula', 'url...
12    {'name': 'Health programs administration and f...
13    {'name': 'Executive agency funding and structu...
14    {'name': 'Drug, alcohol, tobacco use', 'url_na...
15    {'name': 'Emergency communications systems', '...
16    {'name': 'Child safety and welfare', 'url_name...
17    {'name': 'Juvenile crime and gang violence

In [16]:
pd.DataFrame(congress.members.bills(hillary)['bills']).head()

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,None,s211-111,s,https://api.propublica.org/congress/v1/111/bil...,Senate Finance,111,https://www.congress.gov/bill/111th-congress/s...,61,"{'R': 16, 'D': 43, 'ID': 1, 'I': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,Calling for 2-1-1 Act of 2009,None
1,None,s182-111,s,https://api.propublica.org/congress/v1/111/bil...,Senate Finance,111,https://www.congress.gov/bill/111th-congress/s...,42,"{'D': 41, 'I': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,Paycheck Fairness Act,None
2,None,sres727-110,sres,https://api.propublica.org/congress/v1/110/bil...,,110,https://www.congress.gov/bill/110th-congress/s...,3,{'D': 3},None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A resolution honoring the victims of the bombi...,None
3,None,sres720-110,sres,https://api.propublica.org/congress/v1/110/bil...,"Senate Health, Education, Labor, and Pensions",110,https://www.congress.gov/bill/110th-congress/s...,2,"{'D': 1, 'R': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A resolution supporting the goals and ideals o...,None
4,None,s19-110,s,https://api.propublica.org/congress/v1/110/bil...,Senate Finance,110,https://www.congress.gov/bill/110th-congress/s...,0,{},None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A bill to amend the Internal Revenue Code of 1...,None
